In [1]:
from pyspark import SparkContext, SparkConf

In [2]:
import json

In [3]:
conf = SparkConf().setAppName('word count').setMaster('local[3]')
sc = SparkContext(conf = conf)

In [4]:
lines = sc.textFile('./data/4300-0.txt,./data/pg*.txt')

In [5]:
stoplist = sc.textFile('./data/stopwords.txt')

In [6]:
punctuations = sc.broadcast(set(".,:;'!?"))

In [ ]:
punctuations.value

In [7]:
broadcastStopList = sc.broadcast(set(stoplist.collect()))

In [8]:
words = lines.flatMap(lambda line : line.split())

In [ ]:
filteredPunctuation = words.filter(lambda word : len(word) > 0 and word[0] not in punctuations.value and word[-1] not in punctuations.value)

In [ ]:
filteredPunctuation = words.map(lambda word : word[1:] if len(word) > 0 and word[0] in punctuations.value else word).map(lambda word : word[:-1] if len(word) > 0 and word[-1] in punctuations.value else word)

In [ ]:
filteredWords = filteredPunctuation.filter(lambda word : word.lower() not in broadcastStopList.value)

In [9]:
filteredWords = words.filter(lambda word : word.lower() not in broadcastStopList.value)

In [10]:
counts = filteredWords.map(lambda word : (word.lower(), 1)).reduceByKey(lambda a, b : a + b)

In [11]:
counts2 = counts.map(lambda x : (x[0][1:], x[1]) if len(x[0]) > 1 and x[0][0] in punctuations.value else x).map(lambda x : (x[0][:-1], x[1]) if len(x[0]) > 1 and x[0][-1] in punctuations.value else x)

In [16]:
output = counts2.filter(lambda x : x[1] > 2 and len(x[0]) > 1).takeOrdered(40, key=lambda x : -x[1])

In [17]:
output

[('not', 8141),
 ('you', 6354),
 ('have', 5146),
 ('no', 3620),
 ('one', 3498),
 ('like', 2253),
 ('more', 2087),
 ('out', 2021),
 ('up', 1831),
 ('man', 1783),
 ('now', 1579),
 ('only', 1555),
 ('must', 1523),
 ('little', 1485),
 ('those', 1447),
 ('good', 1444),
 ('should', 1417),
 ('after', 1379),
 ('great', 1358),
 ('every', 1356),
 ('first', 1318),
 ('own', 1289),
 ('did', 1271),
 ('how', 1266),
 ('see', 1251),
 ('these', 1244),
 ('men', 1233),
 ('over', 1209),
 ('where', 1205),
 ('make', 1196),
 ('upon', 1188),
 ('nor', 1181),
 ('never', 1177),
 ('much', 1167),
 ('time', 1166),
 ('said', 1163),
 ('two', 1142),
 ('old', 1140),
 ('made', 1128),
 ('most', 1114)]

In [18]:
output = dict(output)

In [19]:
with open('sp3.json', 'w') as outfile:
    json.dump(output, outfile)